In [1]:
import os
from pyprojroot import here
os.environ["HF_HOME"] = str(here("cache/huggingface"))
import lightning as L
from importlib import reload
import sys
sys.path.append(str(here()))
from models import model_definitions as MD
import torch
import pandas as pd
reload(MD)
torch.set_float32_matmul_precision('medium')
data_module_3 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_3.parquet")))
data_module_7 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_7.parquet")))
data_module_28 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_28.parquet")))
metrics = MD.MetricsCallback()

def make_trainer():
  return L.Trainer(
    max_epochs=15,
    max_time={"minutes": 10},
    callbacks=[
      # metrics,
    ],
    deterministic=True,
    default_root_dir=here("cache/lightning"),
    enable_checkpointing=True,
    fast_dev_run=False,
    accelerator="auto", 
  )

trainer = make_trainer()
trainer

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


# Distil Bert finetuned (cross entropy loss)

## 3 classes

In [2]:
reload(MD)
distil_3 = MD.DistilBertFinetune(n_emotions=3)
if True:    
  trainer.fit(
    distil_3,
    datamodule=data_module_3,
  )
else:
  distil_3 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_3_14epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_3,
  datamodule=data_module_3,
)
del distil_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss       | BCEWithLogitsLoss                   | 0      | train
3 | f1_standard    | MultilabelF1Score                   | 0      | train
4 | f1_interesting | MultilabelF1Score                   | 0      | train
5 | rmse           | MeanSquaredError                    | 0      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.4808763563632965
   test_expected_ndcg       0.6376350522041321
   test_f1_interesting      0.5227572321891785
    test_f1_standard        0.5801577568054199
        test_ndcg            0.926764190196991
        test_rmse           0.2648407220840454
    test_weighted_mse       0.12682946026325226
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.4808763563632965,
  'test_rmse': 0.2648407220840454,
  'test_ndcg': 0.926764190196991,
  'test_expected_ndcg': 0.6376350522041321,
  'test_f1_standard': 0.5801577568054199,
  'test_f1_interesting': 0.5227572321891785,
  'test_weighted_mse': 0.12682946026325226}]

## 7 classes

In [3]:
reload(MD)
distil_7 = MD.DistilBertFinetune(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_7,
    datamodule=data_module_7,
  )
else:
  distil_7 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_7_14epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_7,
  datamodule=data_module_7
)
del distil_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.28861114382743835
   test_expected_ndcg       0.42011749744415283
   test_f1_interesting      0.2650759816169739
    test_f1_standard        0.24527610838413239
        test_ndcg           0.8845363855361938
        test_rmse           0.1938202828168869
    test_weighted_mse       0.06779281049966812
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.28861114382743835,
  'test_rmse': 0.1938202828168869,
  'test_ndcg': 0.8845363855361938,
  'test_expected_ndcg': 0.42011749744415283,
  'test_f1_standard': 0.24527610838413239,
  'test_f1_interesting': 0.2650759816169739,
  'test_weighted_mse': 0.06779281049966812}]

## 28 classes

In [4]:
reload(MD)
distil_28 = MD.DistilBertFinetune(n_emotions=28)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_28,
    datamodule=data_module_28,
  )
else:
  distil_28 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_28_11epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.12146283686161041
   test_expected_ndcg       0.2757509648799896
   test_f1_interesting      0.09196062386035919
    test_f1_standard        0.09435343742370605
        test_ndcg           0.7979888916015625
        test_rmse           0.1144043579697609
    test_weighted_mse      0.023443082347512245
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.12146283686161041,
  'test_rmse': 0.1144043579697609,
  'test_ndcg': 0.7979888916015625,
  'test_expected_ndcg': 0.2757509648799896,
  'test_f1_standard': 0.09435343742370605,
  'test_f1_interesting': 0.09196062386035919,
  'test_weighted_mse': 0.023443082347512245}]

In [5]:
# test sample
trainer.predict(
  distil_28,
  ["I Love this"],
)

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[2.5113e-04, 8.4079e-03, 9.7061e-01, 1.2001e-03, 1.9860e-04, 6.9306e-04,
          2.3725e-04, 8.4075e-05, 9.8110e-02, 1.1555e-03, 4.5745e-04, 4.7924e-05,
          6.7671e-04, 6.3971e-04, 1.3686e-02, 1.4553e-03, 3.8156e-02, 8.3070e-05,
          8.2244e-03, 4.8894e-05, 1.0917e-05, 6.1640e-04, 4.5274e-04, 6.1218e-05,
          3.2722e-04, 1.6859e-03, 4.0611e-05, 1.1387e-05]])]

In [6]:
del distil_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')

# Distil Bert finetuned (DCG loss)

## 3 classes

In [7]:
reload(MD) # DistilBertFinetuneOnDCG
distil_dcg_3 = MD.DistilBertFinetuneOnDCG(n_emotions=3)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_3,
    datamodule=data_module_3,
  )
else:
  distil_dcg_3 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_3_10epchs.ckpt"),
    n_emotions=3,
  ) 
out = trainer.test(
  distil_dcg_3,
  datamodule=data_module_3,
)
del distil_dcg_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.6395519375801086
   test_expected_ndcg       0.6316289305686951
   test_f1_interesting      0.32245755195617676
    test_f1_standard        0.19839736819267273
        test_ndcg           0.9285137057304382
        test_rmse           0.3531358540058136
    test_weighted_mse       0.28144562244415283
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.6395519375801086,
  'test_rmse': 0.3531358540058136,
  'test_ndcg': 0.9285137057304382,
  'test_expected_ndcg': 0.6316289305686951,
  'test_f1_standard': 0.19839736819267273,
  'test_f1_interesting': 0.32245755195617676,
  'test_weighted_mse': 0.28144562244415283}]

## 7 classes

In [8]:
reload(MD)
distil_dcg_7 = MD.DistilBertFinetuneOnDCG(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_7,
    datamodule=data_module_7,
  )
else:
  distil_dcg_7 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_7_10epchs.ckpt"),
    n_emotions=7,
  )
  
out = trainer.test(
  distil_dcg_7,
  datamodule=data_module_7,
)
del distil_dcg_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.4227442443370819
   test_expected_ndcg       0.4188288152217865
   test_f1_interesting      0.19298937916755676
    test_f1_standard        0.3904416859149933
        test_ndcg           0.8803690671920776
        test_rmse           0.28776201605796814
    test_weighted_mse       0.10244159400463104
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.4227442443370819,
  'test_rmse': 0.28776201605796814,
  'test_ndcg': 0.8803690671920776,
  'test_expected_ndcg': 0.4188288152217865,
  'test_f1_standard': 0.3904416859149933,
  'test_f1_interesting': 0.19298937916755676,
  'test_weighted_mse': 0.10244159400463104}]

## 28 classes

In [9]:
reload(MD)
distil_dcg_28 = MD.DistilBertFinetuneOnDCG(n_emotions=28)
trainer = make_trainer()
if True:
  trainer.fit(
    distil_dcg_28,
    datamodule=data_module_28,
  )
else:
  distil_dcg_28 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_28_10epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_dcg_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.5016834139823914
   test_expected_ndcg       0.2701887786388397
   test_f1_interesting      0.04936551675200462
    test_f1_standard        0.3519485592842102
        test_ndcg           0.7881404161453247
        test_rmse           0.3729221820831299
    test_weighted_mse       0.14503665268421173
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.5016834139823914,
  'test_rmse': 0.3729221820831299,
  'test_ndcg': 0.7881404161453247,
  'test_expected_ndcg': 0.2701887786388397,
  'test_f1_standard': 0.3519485592842102,
  'test_f1_interesting': 0.04936551675200462,
  'test_weighted_mse': 0.14503665268421173}]

In [10]:
# test sample
trainer.predict(
  distil_dcg_28,
  ["I Love this"],
)

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[0.0719, 0.5337, 0.9960, 0.6254, 0.0407, 0.3526, 0.0513, 0.0471, 0.9156,
          0.2220, 0.1124, 0.0560, 0.3360, 0.1101, 0.5335, 0.3847, 0.7742, 0.0280,
          0.6794, 0.0187, 0.0180, 0.1617, 0.1170, 0.0317, 0.2553, 0.4240, 0.0918,
          0.0149]])]

In [11]:
del distil_dcg_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')

# Distil Bert finetuned (Weighed MSE loss)

## 3 classes

In [12]:
reload(MD)
distil_mse_3 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=3)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_3,
    datamodule=data_module_3,
  )
else:
  distil_mse_3 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_3_10epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_mse_3,
  datamodule=data_module_3,
)
del distil_mse_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.49766263365745544
   test_expected_ndcg        0.63802570104599
   test_f1_interesting      0.4825359582901001
    test_f1_standard        0.6878682374954224
        test_ndcg           0.9288696050643921
        test_rmse           0.2759268879890442
    test_weighted_mse        0.117215596139431
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.49766263365745544,
  'test_rmse': 0.2759268879890442,
  'test_ndcg': 0.9288696050643921,
  'test_expected_ndcg': 0.63802570104599,
  'test_f1_standard': 0.6878682374954224,
  'test_f1_interesting': 0.4825359582901001,
  'test_weighted_mse': 0.117215596139431}]

## 7 classes

In [13]:
reload(MD)
distil_mse_7 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_7,
    datamodule=data_module_7,
  )
else:
  distil_mse_7 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_7_10epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_mse_7,
  datamodule=data_module_7,
)
del distil_mse_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.3003412187099457
   test_expected_ndcg       0.4210582971572876
   test_f1_interesting      0.25360995531082153
    test_f1_standard        0.3205142319202423
        test_ndcg           0.8793054819107056
        test_rmse           0.2038750946521759
    test_weighted_mse       0.06491104513406754
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.3003412187099457,
  'test_rmse': 0.2038750946521759,
  'test_ndcg': 0.8793054819107056,
  'test_expected_ndcg': 0.4210582971572876,
  'test_f1_standard': 0.3205142319202423,
  'test_f1_interesting': 0.25360995531082153,
  'test_weighted_mse': 0.06491104513406754}]

## 28 classes

In [14]:
reload(MD)
distil_mse_28 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=28)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_28,
    datamodule=data_module_28,
  )
else:
  distil_mse_28 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_28_10epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_mse_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.12870030105113983
   test_expected_ndcg       0.2742919325828552
   test_f1_interesting      0.10735047608613968
    test_f1_standard         0.160737544298172
        test_ndcg           0.7874660491943359
        test_rmse           0.12162992358207703
    test_weighted_mse       0.02265126444399357
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.12870030105113983,
  'test_rmse': 0.12162992358207703,
  'test_ndcg': 0.7874660491943359,
  'test_expected_ndcg': 0.2742919325828552,
  'test_f1_standard': 0.160737544298172,
  'test_f1_interesting': 0.10735047608613968,
  'test_weighted_mse': 0.02265126444399357}]

In [15]:
# test sample
trainer.predict(
  distil_mse_28,
  ["I Love this"],
)

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[1.9795e-03, 8.3036e-03, 9.8754e-01, 1.0707e-03, 1.5758e-03, 1.0263e-02,
          1.3120e-03, 6.8178e-04, 1.5830e-01, 1.4609e-02, 1.3186e-03, 4.0372e-03,
          3.9815e-03, 7.6343e-04, 1.4916e-02, 1.9344e-03, 6.7266e-02, 4.2415e-04,
          2.5575e-02, 3.2664e-04, 2.8321e-05, 1.7752e-03, 6.6435e-03, 2.4457e-05,
          4.6084e-04, 9.9287e-03, 1.5615e-04, 2.3644e-04]])]

In [16]:
del distil_mse_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')